### Prep:

* Resample all inputs see 1-Resample Input Data.ipynb
* Rasterize fires and organize by time period see 2-RasterizeFirePolygons_OrganizeData.ipynb

### Data Structure:
Data should be structured as follows:

* Independent Vars 2012-2016
    * aet
        * aet-201201.tif
        * ...
        * aet-201612.tif
    * cwd
        * cwd-201201.tif
        * ...
        * cwd-201612.tif
    * ppt
        * ppt-201201.tif
        * ...
        * ppt-201612.tif
    * ...
* Independent Vars 2017-2021
    * ...
* Dependent Var 2012-2016
    * Fire-2012-2016
* Dependent Var 2017-2021
    * Fire-2017-2021

In [1]:
import tsraster.prep as tr
import tsraster.random as random
import tsraster.model as model
import tsraster.calculate as ca
import numpy as np

import tsraster.model  as md
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from math import ceil
from tsraster.prep import set_df_mindex
import copy

In [10]:
########################### 

In [11]:
#construct poisson disk mask, which masks out all pixels not selected by raster mask
'''
Create raster of cells to be selected (populated as ones) in a raster of background value zero

:param raster_mask: name of raster mask - provides dimensions for subsample, and also masks unusable areas - 
        Remaining sample area is assumed to be contiguous
:param outFile: path and name of output mask consisting of a rater image with values of 1 for selected pixels, 
        and 0 for all other pixels
:param k: number of attempts to select a point around each reference point before marking it as inactive
:param r: minimum distance (in raster cells) between selected points 
'''


rasterMask = random.Poisson_Subsample(raster_mask = r"../Data\Examples\buffer\StatePoly_buf.tif",
                                      outFile = r"../Data\Examples\diskTest.tif",
                                      k=50, 
                                      r=10)[0]

In [12]:
#combine all data that is invariant over time into single dataFrame 
#(e.g., elevation, rate of lightning strikes, fire station locations)

'''Combines set of rasters to single dataFrame based on csv that lists all desired files 
    and column names to use in dataframe for data corresponding to each raster
    
    param csvPath: path to csv of filepaths (in column "FilePath") and desired label names for values in each raster (in column "DataName")
    param outPath: path to ouput file name and location

 '''

invar_Data = tr.multi_image_to_dataframe("C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Invar_FileList.csv", "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/")
invar_Data.head()

,pixel_id,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev
0,0,-1,121,78,-1,-1,0,-15
1,1,-1,-23,-66,-1,-1,0,-15
2,2,-1,95,49,-1,-1,0,-15
3,3,-1,-38,-88,-1,-1,0,-15
4,4,-1,91,34,-1,-1,0,-15


In [13]:
#conduct climate feature extraction across all years of interest


'''Extracts summary statistics(features) from multiYear datasets within moving window, across years
    Outputs a series of annual dataFrames as CSV files
'''

#param startYear: year on which to start feature extraction
#param endYear: year on which to end feature extraction
#param featureData_Path: file path to data from which to extract features
#param feature_params: summary statistics(features) to extract from data within each window
#param invar_Data: year-invariate data to join with extracted feature data on an annual scale
#param out_Path: file path to location at which extracted features should be output as a csv
#param window_length: length of window within which to extract features
#param window_offset: number of years by which features pertaining to each year are offset from that year
#param mask:  mask to apply to data prior to feature extraction

ca.multiYear_Window_Extraction(1980, 2016, "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/",
                    feature_params = {"mean": None,"maximum": None}, 
                    invar_Data = invar_Data,
                    out_Path = 'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/',
                        mask = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Examples/buffer/StatePoly_buf.tif" ,
                    window_length = 3,
                    window_offset = 0)

df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:07<00:00, 13.85s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1980_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1980_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:07<00:00, 13.95s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1981_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1981_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


...

features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2014_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2014_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:07<00:00, 13.86s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2015_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2015_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:06<00:00, 13.56s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2016_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2016_3_prev_offset0.tiff


In [ ]:
#Assembly of all explanatory variables into annual dataFrames

'''merge additional annually repeating data into feature data, as well as time-invariant data
    Produces annual dataFrames consisting of all explanatory variables that may be incorporated into model
        (Consisting of features extracted from climate data in preceding years, 
        annually repeating data such as estimated housing density,
        and time-invariant data such as rate of lightning strikes or local elevation)
        
    param startYear: year on which to start feature extraction
    param endYear: year on which to end feature extraction
    param other_Data_prefixList: list of file path and portion of filename preceding year for additional Data
    param feature_Data_suffixList: portion of feature data file name that follows year for additional data
    param dataNameList: list of intended data names for additional data
    param outPath: filepath for folder in which the output will be placed
'''
    


tr.annual_Data_Merge(startYear = 1980, 
                     endYear = 2016, 
                  feature_path = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/", 
                 invarData_csvPath = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Invar_FileList.csv",
                     other_Data_prefixList = ["C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/SERGOM_Housing/Interpolated/bhc"],
                 other_Data_suffixList = ["linreg.tif"],
                 dataNameList = ["Housing_Density"],
                 outPath = 'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/')

In [ ]:
#Conversion of fire data into annual dataframes

#converts annual fire raster data into annual CSV files, does some minor reformatting for downstream processing
'''convert annual fire data rasters into annual dataFrames, export as .CSV files
    also does some minor reformatting to prevent problems with downstream processing
'''
    
    #param startYear: year on which to start feature extraction
    #param endYear: year on which to end feature extraction
    #param file_Path: path to target data files (fire data)
    #outPath: filepath for folder in which the output will be placed: 



tr.target_Data_to_csv_multiYear(1980, 2016,
                            "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Fires/Rasters/",
                            'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/')

In [ ]:
#Mask All data files (combined_data and target_data) using output of Poisson Disk Masking (or other desired mask)

'''masks annual datasets (combined and fire), and saves the resulting masked data files as CSV files
    Also returns concatenated multiYear files (combined and fire) for subsequent processing - adds in 'year' as a data column
'''
    #param startYear: year on which to begin
    #param endYear: year on which to end
    #param DataLists: csv of files to pull, with the year is the index, 
    #       "combined_Data_Filepaths" as the column of combined data filepaths, and
    #       "target_Data_filePaths" as the column of the target data(i.e. fire) filepaths.
    #param maskFile: filepath to data file used for masking
    #outPath: filepath for folder in which the output will be placed


combined_Data, target_Data = tr.multiYear_Mask(1980, 2016, 
                                               'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/', 
                                               r"../Data\Examples\diskTest.tif", 
                                               "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/")

In [4]:
#temporary for testing only - read in combined & target data 

#combined_Data = pd.read_csv('C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/CD_1980_Masked_2016.csv')
#target_Data = pd.read_csv('C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/TD_1980_Masked_2016.csv')

In [5]:
#Conduct elastic net regularization

'''Conduct elastic net regressions on data, with k-fold cross-validation conducted independently 
      across both years and pixels. 
      Returns mean model MSE and R2 when predicting fire risk at 
      A) locations outside of the training dataset
      B) years outside of the training dataset
      C) locations and years outside of the training dataset
    Returns a list of objects, consisting of:
    0: Combined_Data file with testing/training groups labeled
    1: Target Data file with testing/training groups labeled
    2: summary dataFrame of MSE and R2 for each model run
        (against holdout data representing either novel locations, novel years, or both)
    3: list of elastic net models for use in predicting Fires in further locations/years
    4: list of list of years not used in model training for each run
'''
#param combined_Data: explanatory factors to be used in predicting fire risk
#param target_Data: observed fire occurrences
#param varsToGroupBy: list of (2) column names from combined_Data & target_Data to be used in creating randomized groups
#    -spatial grouping variable 1st, temporal (annual) grouping variable 2nd
#param groupVars: list of (2) desired column names for the resulting randomized groups
#param testGroups: number of distinct groups into which data sets should be divided (for each of two variables) 
  


CrossVal_Output = model.elasticNet_2dimTest(combined_Data, target_Data, ["pixel_id", "year"], ["pixel_group", "year_group"], testGroups = [10, 6])


pixels_Years MSE Overall:  -0.0013183072892232158
pixels_Years R2 Overall:  -3854.667374309443


pixels MSE Overall:  4.193279272659186e-06
pixels R2 Overall:  -3234.7698212982264


years MSE Overall:  -0.00023292608382085022
years R2 Overall:  -3841.0399723548207




In [6]:
#Combined_Data with groups
CrossVal_Output[0].head()

,pixel_id,Unnamed: 0,pixel_id.1,time,aet__maximum,aet__mean,cwd__maximum,cwd__mean,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev,Housing_Density,year,pixel_group,year_group
0,135548,135548,135548,197801_198012,130.338806,42.128597,72.294800,37.287411,-1,-110,-42,-1,-1,0,37,0,1980,5,5
1,137458,137458,137458,197801_198012,105.332001,33.296608,90.760002,43.671833,-1,-74,120,-1,-1,0,-124,0,1980,9,5
2,137513,137513,137513,197801_198012,149.053604,50.056412,60.904800,29.341909,-1,-41,113,-1,-1,0,-114,0,1980,4,5
3,138417,138417,138417,197801_198012,134.289200,56.976852,41.080399,19.541698,-1,55,-67,-1,-1,0,5,0,1980,2,5
4,138480,138480,138480,197801_198012,149.436798,44.156055,164.356400,42.745041,-1,33,-16,-1,-1,0,-8,2,1980,2,5


In [7]:
#target Data with groups
CrossVal_Output[1].head()

,pixel_id,value,year,pixel_group,year_group
0,135548,0,1980,5,5
1,137458,0,1980,9,5
2,137513,0,1980,4,5
3,138417,0,1980,2,5
4,138480,0,1980,2,5


In [8]:
# summary MSE and R2 for all runs, against spatially novel, temporally novel, and completely novel data
CrossVal_Output[2]

,Pixels_Years_MSE,Pixels_MSE,Years_MSE,Pixels_Years_R2,Pixels_R2,Years_R2
0,0.000233,0.000269,0.000228,-6216.404318,-2734.097152,-5567.646325
1,-0.000504,0.000310,-0.000468,-2667.337091,-2604.554365,-3111.815410
2,0.000428,-0.000252,0.000051,-3588.999101,-2474.519138,-3086.483234
3,-0.000195,0.000080,0.000492,-3928.646524,-3443.866953,-4569.659501
4,-0.003041,0.000566,-0.001747,-1733.036393,-2087.375903,-2037.279206
5,-0.001339,-0.000015,0.000283,-1930.612464,-3457.065202,-2864.791513
6,-0.001878,0.000267,0.000225,-3719.865216,-3207.306907,-6410.653835
7,-0.000315,0.000257,-0.000623,-1798.431249,-2908.954710,-2978.566323
8,0.000005,0.000248,0.000112,-2463.012318,-3198.546875,-3520.781721
9,-0.000385,0.000024,0.000338,-8778.901216,-2997.972876,-4553.915015


In [9]:
#List of models
CrossVal_Output[3]

[(ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  0.00023266663508525465,
  -6216.404318055572),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  -0.0005035961395247757,
  -2667.3370907897065),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  0.0004284689765455818,
  -3588.999100818164),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='c

In [10]:
#list of years excluded from training data for each model run
CrossVal_Output[4]

[[1990, 1992, 1997, 2003, 2004, 2010],
 [1984, 1987, 2006, 2007, 1981, 2014],
 [2016, 1994, 2002, 2008, 2013, 1983],
 [1986, 1991, 1996, 1998, 1999, 2009],
 [1985, 1988, 1995, 2000, 2005, 2011],
 [1989, 1993, 2012, 2001, 1980, 1982, 2015],
 [1990, 1992, 1997, 2003, 2004, 2010],
 [1984, 1987, 2006, 2007, 1981, 2014],
 [2016, 1994, 2002, 2008, 2013, 1983],
 [1986, 1991, 1996, 1998, 1999, 2009],
 [1985, 1988, 1995, 2000, 2005, 2011],
 [1989, 1993, 2012, 2001, 1980, 1982, 2015],
 [1990, 1992, 1997, 2003, 2004, 2010],
 [1984, 1987, 2006, 2007, 1981, 2014],
 [2016, 1994, 2002, 2008, 2013, 1983],
 [1986, 1991, 1996, 1998, 1999, 2009],
 [1985, 1988, 1995, 2000, 2005, 2011],
 [1989, 1993, 2012, 2001, 1980, 1982, 2015],
 [1990, 1992, 1997, 2003, 2004, 2010],
 [1984, 1987, 2006, 2007, 1981, 2014],
 [2016, 1994, 2002, 2008, 2013, 1983],
 [1986, 1991, 1996, 1998, 1999, 2009],
 [1985, 1988, 1995, 2000, 2005, 2011],
 [1989, 1993, 2012, 2001, 1980, 1982, 2015],
 [1990, 1992, 1997, 2003, 2004, 2010],
 